In [ ]:
pip install transformers

     |████████████████████████████████| 2.3MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 48.5MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 


In [ ]:
import pandas as pd
import json
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from transformers import ElectraForQuestionAnswering
model = ElectraForQuestionAnswering.from_pretrained("deepset/electra-base-squad2")
from transformers import  pipeline
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from transformers import ElectraTokenizerFast
tokenizer = ElectraTokenizerFast.from_pretrained("deepset/electra-base-squad2")



The following cell converts a json file into dataframe. The content of our sQuad json file is converted into dataframe in order to visualise it in a better way. This dataframe will be having the features with context, questions, answer start and the text.

In [ ]:
def json_to_dataframe(file):
    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt

    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

In [ ]:
df=json_to_dataframe('/content/train-v2.0.json')
df.head(5)

,Id,title,context,question,ans_start,text
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,269,in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,207,singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,526,2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,166,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,276,late 1990s


The following fuction segregates content of our json file into 

*   **Context**, which has the paragraph content from which questions are framed.
*   **Questions**, will be having the specific question.
*   **Answers**, answer for that particular question.






In [ ]:
def read_squad(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa['answers']:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_squad('/content/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('/content/dev-v2.0.json')

The following function creates a new feature as answer end which will be number signifying the end position of answer. Which will be used in training our model.

In [ ]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [ ]:
add_end_idx(train_answers, train_contexts)
# add_end_idx(val_answers, val_contexts)

Tokenizing all the contexts, questions and answers from dataset

In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
# val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

A tokenizer is in charge of preparing the inputs for a model. 
Tokenizing (splitting strings in sub-word token strings), converting tokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).

In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


In [ ]:
add_token_positions(train_encodings, train_answers)

Preparing the input to our pytorch model

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = SquadDataset(train_encodings)

Training our model of 2 epochs and printing the loss

In [ ]:
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(2):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 1: 100%|██████████| 7/7 [03:31<00:00, 30.18s/it, loss=1.79]


Testing our model with a random question and context, dev file will be used in later stages

In [ ]:
# a) Get predictions
nlp = pipeline('question-answering', model, tokenizer=tokenizer)
text='The Times of India is an Indian English-language daily newspaper and digital news media owned and managed by The Times Group. It is the third-largest newspaper in India by circulation and largest selling English-language daily in the world.'
question='What is Times of India ?'
result=nlp(question,text)
result['answer']

'Indian English-language daily newspaper'

Following 5 cells are executed to push our model into model hub of hugging faces AI community

In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!transformers-cli login

2021-06-02 06:35:16.939492: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: luka89661@gmail.com
Password: 
Login successful
Your token: VSLXIYpMWzpFFJUHVkPSnQRoHASWXsFErLquDZrHrJOcTncPHcqrdRNGFXlpLZwLddtYHeHCEcfPiGevYziRXDQByUsKNeReZtsfsClBigqagsMUlWGfIUmyPuzlsSMG 

Your token has been saved to /root/.huggingface/tok

In [ ]:
!git config --global user.email "luka89661@gmail.com"
!git config --global user.name "orange"

In [ ]:
model.push_to_hub("electra-Question-answer")

In [ ]:
tokenizer.push_to_hub("electra-Question-answer")

Predicting answers for the dev file and testing text file and storing details in the csv file with the name dev_predictions and testing_answers respectively.

In [ ]:
dev_df=pd.DataFrame(columns=['CONTEXT','QUESTION','ACTUAL_ANSWER','PREDICTED_ANSWER','START','END'])
for i in range(len(val_contexts)):
  context=val_contexts[i]
  question=val_questions[i]
  result=nlp(val_questions[i],val_contexts[i])
  dev_df=dev_df.append({'CONTEXT':context,'QUESTION':question,'ACTUAL_ANSWER':val_answers[i]['text'],'PREDICTED_ANSWER':result['answer'],'START':result['start'],'END':result['end']},ignore_index=True)

In [ ]:
dev_df.to_csv('dev_predictions.csv',index=False)

In [ ]:
text=pd.read_csv('/content/test_set_questions.txt',header=None)

In [ ]:
text.rename({0:'CONTEXT',1:'QUESTION'}, axis=1, inplace=True)

In [ ]:
text_df=pd.DataFrame(columns=['CONTEXT','QUESTION','ANSWER','START','END'])
for i in range(len(text)):
  context=text.CONTEXT[i]
  question=text.QUESTION[i]
  result=nlp(text.QUESTION[i],text.CONTEXT[i])
  text_df=text_df.append({'CONTEXT':context,'QUESTION':question,'ANSWER':result['answer'],'START':result['start'],'END':result['end']},ignore_index=True)
  print(i)

In [ ]:
text_df.to_csv('testing_answers.csv', index=False)